In [56]:
import os
import tempfile
import subprocess
import sys

def install_playwright():
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "playwright"], check=True)
        subprocess.run([sys.executable, "-m", "playwright", "install", "chromium"], check=True)
        return True
    except:
        return False

def save_page_as_image(page_num):
    ar = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/quran-uthmani").json()["data"]["ayahs"]
    en = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/en.sahih").json()["data"]["ayahs"]
    
    # Arabic + translation
    arabic_translation = []
    for a,e in zip(ar,en):
        arabic_translation.append(f"<div style='margin:10px 0'><div style=\"direction:rtl;text-align:right;font-family:'Amiri','Scheherazade New','Times New Roman',serif;font-size:24px;line-height:2.2\">{a['text']} <span style='color:#d4af37;font-size:.8em;margin-left:10px'>{a['numberInSurah']}</span></div><div style=\"font-family:'Inter',-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Arial,sans-serif;font-size:16px;line-height:1.6;color:#eee\">{e['text']}</div></div>")
    
    # Save Arabic + translation
    translation_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <link href="https://fonts.googleapis.com/css2?family=Amiri:wght@400;700&family=Scheherazade+New:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {{ margin: 0; padding: 20px; background: #1e293b; color: white; }}
        </style>
    </head>
    <body>
        <div style="background:#1e293b;padding:16px;border-radius:10px;width:800px">{''.join(arabic_translation)}</div>
    </body>
    </html>
    """
    
    # Create temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix='_translation.html', delete=False, encoding='utf-8') as f:
        f.write(translation_html)
        translation_file = f.name
    
    translation_output = f"quran_page_{page_num}_arabic_translation.png"
    
    try:
        # Save Arabic + translation
        result = subprocess.run([
            sys.executable, "-c", f"""
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto('file://{translation_file}')
        await page.screenshot(path='{translation_output}', full_page=True)
        await browser.close()

asyncio.run(main())
"""
        ], capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"✅ Arabic + translation saved as {translation_output}")
            os.unlink(translation_file)
        else:
            print(f"❌ Error: {result.stderr}")
    except Exception as e:
        print(f"❌ Error: {e}")

page_num = int(input("Enter page number (1-604): "))
save_page_as_image(page_num)


✅ Arabic + translation saved as quran_page_77_arabic_translation.png
